# SQL agent

## Setup

### Imports

In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..\\app"))
if project_root not in sys.path:
    sys.path.append(project_root)

In [6]:
from llm_app.databases.external_db import get_db
from llm_app.test_datasets import TestData
from llm_app.prompts import PromptGenerator
from llm_app.examples import FEW_SHOT_EXAMPLES

from llm_app.agents import OllamaFunctionsSQLAgent

from langsmith import Client

from langchain_community.chat_models.ollama import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings


from langchain_experimental.llms.ollama_functions import OllamaFunctions

### LangSmith

In [ ]:
os.environ["LANGCHAIN_PROJECT"] = "text2sql"
client = Client()

### Load models

In [ ]:
llama3_embeddings = OllamaEmbeddings(
    model="llama3:8b", 
    temperature=0
)

llama3__chat = ChatOllama(
    model="llama3:instruct", 
    temperature=0
)

llama3__with_functions = OllamaFunctions(
    model="llama3:instruct",
    format="json",
    temperature=0
)

### Connect to DB with Readonly role

In [ ]:
db = get_db()

#### Check connection

In [ ]:
db.run("select * from passenger")

-----
## Testing an agent

In [ ]:
prompt_generator = PromptGenerator().set_example_selector(
    examples=FEW_SHOT_EXAMPLES,
    embedding_llm=llama3_embeddings,
)

agent = OllamaFunctionsSQLAgent(
    ollama_functions=llama3__with_functions,
    chat_llm=llama3__chat,
    prompt_generator=prompt_generator,
    db=db
)

executor = agent.get_runnable()
res = executor.invoke(TestData.QUESTIONS[0])

In [ ]:
print(res.return_values["output"])

In [ ]:
batch_res = executor.batch(TestData.QUESTIONS[:2])

In [ ]:
print(batch_res[0].return_values["output"])
print(batch_res[1].return_values["output"])